In [1]:
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
from keras import backend as K
import numpy as np
import pylab as plt
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, CSVLogger
from data_for_UNET import load_train_data, load_test_data

Using TensorFlow backend.


In [2]:
#from LoadDataHelper import *

In [3]:
#myLoader = CTADataLoader('E:/CTAData/ALLDataINLabPCD')
#imgs_train,labs_train = myLoader.loadData()#这里不建议使用Loader，因为没有做窗宽窗位的调整

def pre_process():
    imgs_train, labs_train = load_train_data()
    imgs_train = np.transpose(imgs_train, (2, 0, 1, 3))

    labs_train = np.transpose(labs_train, (2, 0, 1, 3))

    imgs_train = imgs_train.astype(np.float32)
    labs_train=labs_train.astype(np.float32)
    return imgs_train,labs_train

#load 原始数据
imgs_train,labs_train = pre_process()

#load cnn的结果
#imgs_train = np.load('cnnPredectResutl.npy')
#labs_train = np.load('cnnPredectLabels.npy')







In [4]:
labs_train.shape


(582, 512, 512, 1)

In [5]:
def dice_coef(y_true,y_pred,epsilon=1e-8):
    axes = tuple(range(1,len(y_pred.shape)-1))
    intersection = 2.*K.sum(y_true*y_pred,axes)
    return (intersection + epsilon)/(K.sum((K.square(y_true)+K.square(y_pred)),axes) + epsilon)

def dice_coef_loss(y_ture,y_pred):
    return 1-dice_coef(y_ture,y_pred) 

In [6]:
seq = Sequential()
seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   input_shape=(None, 512, 512, 1),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
               activation='sigmoid',
               padding='same', data_format='channels_last'))
seq.compile(loss=dice_coef_loss, optimizer='adadelta')

W1009 20:24:37.871079 13112 deprecation_wrapper.py:119] From c:\users\administrator\anaconda3\envs\kerasenv\lib\site-packages\keras\backend\tensorflow_backend.py:68: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1009 20:24:37.886707 13112 deprecation_wrapper.py:119] From c:\users\administrator\anaconda3\envs\kerasenv\lib\site-packages\keras\backend\tensorflow_backend.py:507: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1009 20:24:37.886707 13112 deprecation_wrapper.py:119] From c:\users\administrator\anaconda3\envs\kerasenv\lib\site-packages\keras\backend\tensorflow_backend.py:3831: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1009 20:24:38.042977 13112 deprecation_wrapper.py:119] From c:\users\administrator\anaconda3\envs\kerasenv\lib\site-packages\keras\backend\tensorflow_backend.py:167: The name tf.get_default_session is deprecated. Please use tf.compat.v

def generate_movies(n_samples=1200, n_frames=15):
    row = 80
    col = 80
    noisy_movies = np.zeros((n_samples, n_frames, row, col, 1), dtype=np.float)
    shifted_movies = np.zeros((n_samples, n_frames, row, col, 1),
                              dtype=np.float)
    for i in range(n_samples):
        # Add 3 to 7 moving squares
        n = np.random.randint(3, 8)
        for j in range(n):
            # Initial position
            xstart = np.random.randint(20, 60)
            ystart = np.random.randint(20, 60)
            # Direction of motion
            directionx = np.random.randint(0, 3) - 1
            directiony = np.random.randint(0, 3) - 1
            # Size of the square
            w = np.random.randint(2, 4)
            for t in range(n_frames):
                x_shift = xstart + directionx * t
                y_shift = ystart + directiony * t
                noisy_movies[i, t, x_shift - w: x_shift + w,
                             y_shift - w: y_shift + w, 0] += 1
                # Make it more robust by adding noise.
                # The idea is that if during inference,
                # the value of the pixel is not exactly one,
                # we need to train the network to be robust and still
                # consider it as a pixel belonging to a square.
                if np.random.randint(0, 2):
                    noise_f = (-1)**np.random.randint(0, 2)
                    noisy_movies[i, t,
                                 x_shift - w - 1: x_shift + w + 1,
                                 y_shift - w - 1: y_shift + w + 1,
                                 0] += noise_f * 0.1

                # Shift the ground truth by 1
                x_shift = xstart + directionx * (t + 1)
                y_shift = ystart + directiony * (t + 1)
                shifted_movies[i, t, x_shift - w: x_shift + w,
                               y_shift - w: y_shift + w, 0] += 1
    # Cut to a 40x40 window
    noisy_movies = noisy_movies[::, ::, 20:60, 20:60, ::]
    shifted_movies = shifted_movies[::, ::, 20:60, 20:60, ::]
    noisy_movies[noisy_movies >= 1] = 1
    shifted_movies[shifted_movies >= 1] = 1
    return noisy_movies, shifted_movies

In [7]:
#noisy_movies, shifted_movies = generate_movies(n_samples=1200, n_frames = 1)
noisy_movies = imgs_train[::,np.newaxis, ::, ::,::]
shifted_movies = labs_train[::,np.newaxis, ::, ::,::]

In [8]:
type(noisy_movies)

numpy.ndarray

In [9]:
noisy_movies.shape

(582, 1, 512, 512, 1)

In [10]:
noisy_movies.max()

1.0

#将数据保存到一个指定的目录下
noisy_moviesPath = 'D:/Data/conv_lstm_data/noisy_movies/'
shifted_moviesPath = 'D:/Data/conv_lstm_data/shifted_movies/'
dataPath = 'D:/Data/conv_lstm_data/data/'
#通过imageJ观察保存下载的图片
#获取帧数
frameNum = noisy_movies.shape[0]
#写个for循环，保存每一帧
for i in range(15):    
    tmpNoiseImage = noisy_movies[1, i, ::, ::, 0]#取的是每一个视频中的第1帧
    tmpShiftedImage = shifted_movies[1, i, ::, ::, 0]
    # Let's look at the results.
    plt.figure(figsize=(16,8))
    plt.subplot(1,2,1)
    plt.imshow(tmpNoiseImage,cmap="magma")
    plt.axis('off') # 不显示坐标轴
    plt.title('NoiseImage');
    plt.subplot(1,2,2)
    plt.imshow(tmpShiftedImage,cmap="magma")
    plt.axis('off') # 不显示坐标轴
    plt.title('ShiftedImage');
    plt.savefig(dataPath+'%i_sourceVideo.png' % (i + 1))
    
def showArray(array,axis=0):
    frameNum = array.shape[axis]
    for i in range(frameNum):    
        tmpNoiseImage = array[0, i, ::, ::, 0]#取的是每一个视频中的第1帧
        plt.figure(figsize=(16,8))
        plt.subplot(1,1,1)
        plt.imshow(tmpNoiseImage,cmap="magma")
        plt.axis('off') # 不显示坐标轴
        plt.title('Image');
        which = 1004
track = noisy_movies[which][:7, ::, ::, ::]
aa = track[np.newaxis, ::, ::, ::, ::]
aa.shape
    


weightStoreFatherPath = './LSTMWeights/'
weightStorePath=weightStoreFatherPath + 'weight.hdf5'
model_checkpoint = ModelCheckpoint(weightStorePath, verbose=1, monitor='val_loss', save_best_only=True)
csv_logger = CSVLogger('./LSTMWeights/log.csv', append=True, separator='\t')

## 训练网络

In [ ]:
seq.fit(noisy_movies, shifted_movies, batch_size=10,
        epochs=100, validation_split=0.05,callbacks=[model_checkpoint,csv_logger])

W1009 12:04:25.722305 15260 deprecation.py:323] From c:\users\administrator\anaconda3\envs\kerasenv\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 552 samples, validate on 30 samples
Epoch 1/100
552/552 [==============================] - 97s 175ms/step - loss: 0.9971 - val_loss: 0.9985

Epoch 00001: val_loss improved from inf to 0.99851, saving model to ./LSTMWeights/weightCNNLSTM.hdf5
Epoch 2/100
552/552 [==============================] - 87s 158ms/step - loss: 0.9936 - val_loss: 0.9998

Epoch 00002: val_loss did not improve
Epoch 3/100
552/552 [==============================] - 87s 158ms/step - loss: 0.9645 - val_loss: 1.0000

Epoch 00003: val_loss did not improve
Epoch 4/100
552/552 [==============================] - 87s 158ms/step - loss: 1.0000 - val_loss: 1.0000

Epoch 00004: val_loss did not improve
Epoch 5/100
552/552 [==============================] - 87s 158ms/step - loss: 1.0000 - val_loss: 1.0000

Epoch 00005: val_loss did not improve
Epoch 6/100
552/552 [==============================] - 87s 158ms/step - loss: 1.0000 - val_loss: 1.0000

Epoch 00006: val_loss did not improve
Epoch 7/100
552/552 [=============

552/552 [==============================] - 87s 158ms/step - loss: 1.0000 - val_loss: 1.0000

Epoch 00056: val_loss did not improve
Epoch 57/100
552/552 [==============================] - 87s 158ms/step - loss: 1.0000 - val_loss: 1.0000

Epoch 00057: val_loss did not improve
Epoch 58/100
552/552 [==============================] - 87s 158ms/step - loss: 0.9530 - val_loss: 0.8366

Epoch 00058: val_loss improved from 0.93179 to 0.83665, saving model to ./LSTMWeights/weightCNNLSTM.hdf5
Epoch 59/100
552/552 [==============================] - 87s 158ms/step - loss: 0.4413 - val_loss: 0.8326

Epoch 00059: val_loss improved from 0.83665 to 0.83263, saving model to ./LSTMWeights/weightCNNLSTM.hdf5
Epoch 60/100
552/552 [==============================] - 87s 158ms/step - loss: 0.3600 - val_loss: 0.9018

Epoch 00060: val_loss did not improve
Epoch 61/100
552/552 [==============================] - 87s 158ms/step - loss: 0.3885 - val_loss: 0.9999

Epoch 00061: val_loss did not improve
Epoch 62/100
55

### 将训练好的模型保存起来

In [25]:
noisy_movies.shape

(582, 1, 512, 512, 1)

### 使用训练好的网络进行预测

In [11]:

import cv2
PredectionsImagesPath = './LSTMResults/PredectionsImages/'
MaskImagesPath ='./LSTMResults/LabelsImages/'
# matplotlib.image.imsave(PredectionsImagesPath+'name.png', oneimage)
weightStorePath =  './LSTMWeights/'+'weight.hdf5'
seq.load_weights(weightStorePath)
for i in range(noisy_movies.shape[0]):
    which = i
    track = noisy_movies[which][::, ::, ::, ::]    
    new_pos = seq.predict(track[np.newaxis, ::, ::, ::, ::])
    track2 = shifted_movies[which][::, ::, ::, ::]
    
    
    cv2.imwrite(PredectionsImagesPath+'%i.png'%(which+1), new_pos[0, 0, ::, ::, 0])
    cv2.imwrite(MaskImagesPath+'%i.png'%(which+1), track2[0, ::, ::, 0])
    
    
    
    
#     from matplotlib import pyplot as plt
#     fig = plt.figure(figsize=(10, 5))
#     ax = fig.add_subplot(131)
#     ax.text(1, 3, 'LSTM Predictions', fontsize=20)
#     toplot = new_pos[0, 0, ::, ::, 0]
#     plt.imshow(toplot)
#     ax = fig.add_subplot(132)
#     plt.text(1, 3, 'Ground Truth', fontsize=20)
#     toplot = track2[0, ::, ::, 0]
#     plt.imshow(toplot)
#     ax = fig.add_subplot(133)
#     plt.text(1, 3, 'Original Image', fontsize=20)
#     toplot = track[0, ::, ::, 0]
#     plt.imshow(toplot)
#     plt.savefig(dataPath+'%i_animate.png' % (which + 1))
    pass


In [27]:
oneimage = new_pos[0, 0, ::, ::, 0]

In [28]:
oneimage.shape

(512, 512)

In [23]:
new_pos.shape

(1, 1, 512, 512, 1)

In [24]:
track2.shape

(1, 512, 512, 1)